In [ ]:
import numpy as np
import dedalus.public as d3
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger(__name__)
%config InlineBackend.figure_format = 'retina'

In [ ]:
nx = 512
Lx = 1
dealias = 3/2
dtype = np.float64
xcoord = d3.Coordinate('x')
dist = d3.Distributor(xcoord, dtype=dtype)
xbasis = d3.ChebyshevT(xcoord, size=nx, bounds=(0, Lx), dealias=dealias)

lift_basis = xbasis.derivative_basis(1)
lift = lambda A, n: d3.Lift(A, lift_basis, n)

In [ ]:
Pe = 1e2
P = 1/Pe
amp = 0.1

In [ ]:
x = dist.local_grid(xbasis)
x_d = dist.local_grid(xbasis, scale=dealias)


c = dist.Field(name='c', bases=xbasis)
τ1 = dist.Field(name='τ1')
τ2 = dist.Field(name='τ2')

u = dist.Field(name='u')
D = dist.Field(name='D', bases=xbasis)
D0 = dist.Field(name='D0', bases=xbasis)

u['g'] = 1
D['g'] = 1 + amp*np.sin(x/Lx*2*np.pi)
D0['g'] = 0.9 #D['g'] #1+amp*2

In [ ]:
# Substitutions
dx = lambda A: d3.Differentiate(A, xcoord)

# Problem
problem = d3.IVP([c, τ1, τ2], namespace=locals())
problem.add_equation("dt(c)  - P*D0*dx(dx(c)) + lift(τ1, -1) + lift(τ2, -2) = - u*dx(c) + P*(D-D0)*dx(dx(c))")
problem.add_equation("c(x=0) - c(x=Lx) = 0")
problem.add_equation("dx(c)(x=0) - dx(c)(x=Lx) = 0")

In [ ]:
integ = lambda A: d3.Integrate(A, 'x')
def L2_error(f, f_true):
    return (integ(np.abs(f-f_true))).evaluate()['g'][0]

In [ ]:
timestepper = d3.SBDF2
Δx = Lx/nx
Δt_cfl = Δx
safety = 0.5
Δt = safety*Δt_cfl
solver = problem.build_solver(timestepper)
solver.stop_sim_time = 1

In [ ]:
σ = 0.1
x0 = 0.5
c.change_scales(1)
c['g'] = np.cos((x-x0)/Lx*np.pi*2)*np.exp(-(x-x0)**2/(2*σ**2))

fig, ax = plt.subplots()
ax.plot(x, c['g'])
while solver.proceed:
    solver.step(Δt)
    if solver.sim_time % 0.1 < (0 + Δt):
        logger.info("iter = {:d}, Δt = {:.2g}, t = {:.2g}, c_max = {:.2g}".format(solver.iteration, Δt, solver.sim_time, np.max(c['g'])))
        ax.plot(x_d, c['g'], linestyle='dashed', alpha=0.5)
ax.plot(x_d, c['g'])
ax.plot(x_d, D['g'], color='xkcd:dark grey', linestyle='dotted', alpha=0.5)
ax.plot(x_d, D0['g'], color='xkcd:dark red', linestyle='dashed', alpha=0.5)

In [ ]:
timestepper = d3.RK222
Δx = Lx/nx
Δt_cfl = Δx
safety = 0.5
Δt = safety*Δt_cfl

c_max = []
c_profiles = []
Ds = []

for system in ['subsystems', 'solvers']:
    logging.getLogger(system).setLevel(logging.WARNING)

for D0_amp in np.geomspace(0.6,5, num=20):
    logger.info("D0 = {:.3g}".format(D0_amp))
    D0['g'] = D0_amp
    solver = problem.build_solver(timestepper)
    solver.stop_sim_time = 1

    c.change_scales(1)
    c['g'] = np.cos((x-x0)/Lx*np.pi*2)*np.exp(-(x-x0)**2/(2*σ**2))

    while solver.proceed:
        solver.step(Δt)
    c_max.append(np.max(c['g']))
    Ds.append(np.mean(D0['g']))
    c_profiles.append(c.copy())

for D0_amp in [D['g']]:
    D0['g'] = D0_amp
    solver = problem.build_solver(timestepper)
    solver.stop_sim_time = 1

    c.change_scales(1)
    c['g'] = np.cos((x-x0)/Lx*np.pi*2)*np.exp(-(x-x0)**2/(2*σ**2))

    while solver.proceed:
        solver.step(Δt)
    c_truth = c.copy()
    D_truth = np.mean(D0['g'])

In [ ]:
L2 = []
for c_i in c_profiles:
    L2.append(L2_error(c_i, c_truth))
    
fig, ax = plt.subplots()
ax.scatter(Ds, L2)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-10, 1e-1)
ax.axvline(x=D_truth, linestyle='dashed')
ax.axvline(x=D_truth+amp, linestyle='dashed')
ax.axvline(x=D_truth-amp, linestyle='dashed')

ax.set_xlabel('<D0>')
ax.set_ylabel(r'$|c-c_t|$')

In [ ]:
fig, ax = plt.subplots()
for i, c_i in enumerate(c_profiles):
    if L2[i] < 1:
        ax.plot(x_d, c_i['g'])

In [ ]:
Δx = Lx/nx
Δt_cfl = Δx
safety = 0.5
Δt = safety*Δt_cfl

dataset = {}

for system in ['subsystems', 'solvers']:
    logging.getLogger(system).setLevel(logging.WARNING)

for timestepper in [d3.SBDF1, d3.SBDF2, d3.SBDF3, d3.SBDF4, d3.RK222, d3.RK443]:
    c_max = []
    c_profiles = []
    Ds = []
    logger.info(timestepper.__name__)
    for D0_amp in np.geomspace(0.6,5, num=20):
        D0['g'] = D0_amp
        solver = problem.build_solver(timestepper)
        solver.stop_sim_time = 1

        c.change_scales(1)
        c['g'] = np.cos((x-x0)/Lx*np.pi*2)*np.exp(-(x-x0)**2/(2*σ**2))

        while solver.proceed:
            solver.step(Δt)
        c_max.append(np.max(c['g']))
        Ds.append(np.mean(D0['g']))
        c_profiles.append(c.copy())
   
    for D0_amp in [D['g']]:
        D0['g'] = D0_amp
        solver = problem.build_solver(timestepper)
        solver.stop_sim_time = 1

        c.change_scales(1)
        c['g'] = np.cos((x-x0)/Lx*np.pi*2)*np.exp(-(x-x0)**2/(2*σ**2))

        while solver.proceed:
            solver.step(Δt)
        c_truth = c.copy()
        D_truth = np.mean(D0['g'])
        
    data = {}
        
    data['c_max'] = c_max
    data['c_profiles'] = c_profiles
    data['Ds'] = Ds
    data['c_truth'] = c_truth
    data['D_truth'] = D_truth
    
    dataset[timestepper] = data
 

In [ ]:
fig, ax = plt.subplots()
for key in dataset:
    Ds = dataset[key]['Ds']
    c_truth = dataset[key]['c_truth']
    L2 = []
    for c_i in dataset[key]['c_profiles']:
        L2.append(L2_error(c_i, c_truth))
    
    ax.scatter(Ds, L2, label=key.__name__)
    
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-7, 1e-1)
ax.axvline(x=D_truth, linestyle='dashed')
ax.axvline(x=D_truth+amp, linestyle='dashed')
ax.axvline(x=D_truth-amp, linestyle='dashed')

ax.set_xlabel('<D0>')
ax.set_ylabel(r'$|c-c_t|$')

In [ ]:
truth = d3.RK443

fig, ax = plt.subplots()
for key in dataset:
    Ds = dataset[key]['Ds']
    c_truth = dataset[truth]['c_truth']
    L2 = []
    for c_i in dataset[key]['c_profiles']:
        L2.append(L2_error(c_i, c_truth))
     
    ax.scatter(Ds, L2, label=key.__name__)
    
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-8, 1e-3)
ax.axvline(x=D_truth, linestyle='dashed')
ax.axvline(x=1.25*D_truth, linestyle='dashed')
ax.axvline(x=0.75*D_truth, linestyle='dashed')

ax.set_xlabel('<D0>')
ax.set_ylabel(r'$|c-c_t|$')